This file adds number of stations of each cluster type.

In [ ]:
import pandas as pd
import us
from tqdm import tqdm
import numpy as np
import category_groupings_250403 as cg

In [ ]:
root = ''
path = root + 'Data/'
path_US_data = root + 'Data/geodata/'
result_path = root + 'final_data/'
path_IRA = root + 'Data/IRA/1.0-shapefile-codebook/usa/'

# Read data

In [ ]:
# Read BG-level data (21_compile_BGlevel)
label = '250702'
df_BG_US = pd.read_csv(result_path + 'BGlevel/21_level_BG_US_compiled_'+label+'.csv',index_col=0)
df_BG_US.index = df_BG_US.index.astype(str).str.zfill(12)
df_BG_US.head(3)

In [ ]:
# Read station level data
df_stations_wlabels = pd.read_csv(result_path + 'stationlevel/20d_level_stations_US_compiled_250415_wlabels.csv', index_col=0)
df_stations_wlabels['BGFP'] = df_stations_wlabels['BGFP'].astype(str).str.zfill(12)
df_stations_wlabels.head()

# Count number of stations by label

In [ ]:
# Get label names for k=8
# Following https://github.com/SiobhanPowell/charging-location-POIs/blob/analysis/Analysis/figure4-6_clusteringplots.ipynb (2025/05/30)
label_mapping = {}
label_mapping['Kmeans_nc8'] = {0:'University/college campuses',
                               1:'Weekday daytime, short stops',
                               2:'City neighborhoods',#Medium density city',
                               3:'Weekday daytime, long stops',
                               4:'Gas stations, very short stops',
                               5:'Weekend evening recreation',
                               6:'High density city',
                               7:'Hotels'}

In [ ]:
# Get number of stations per label and BG
no_clusters = 'Kmeans_nc8'
df_labels_by_BG = df_stations_wlabels.groupby(['BGFP',no_clusters]).size().reset_index().pivot(index='BGFP', columns=no_clusters, values=0).fillna(0).astype(int)
new_cols = []
for col in df_labels_by_BG.columns:
    # df_labels_by_BG.rename(columns={col: no_clusters + '_' + str(col)}, inplace=True)
    new_col = label_mapping['Kmeans_nc8'][col].replace(' ','_')
    new_col = new_col.replace(',','').replace('/','_').replace('-','_')
    new_col = 'no_stations_k8_' + new_col
    df_labels_by_BG.rename(columns={col: new_col}, inplace=True)
    print(new_col)
    new_cols.append(new_col)
df_labels_by_BG

In [ ]:
# Merge columns
df_BG_US_wlabels = df_BG_US.merge(df_labels_by_BG, how='left', left_on='BGFP', right_index=True)
for key in new_cols:
    df_BG_US_wlabels[key] = df_BG_US_wlabels[key].fillna(0).astype(int)
df_BG_US_wlabels.head(3)

# Save

In [ ]:
# Save
label = '250702'
df_BG_US_wlabels.to_csv(result_path + 'BGlevel/21c_level_BG_US_compiled_wlabels_'+label+'.csv')